# 1. Main script
## Main idea:
- Each method is implemented for a specific sample and a specific sub-path (usually sub-path 0 and sample 0). 
- In the end, each methodology will be applied to each sample and subpathway using iterative methods, because of the large number of data available.

In [ ]:
import pandas as pd
import numpy as np
import math

# 2. Read data and make them easier to understand

## 2.1. GSE2034

In [ ]:
raw_gse2034_df = pd.read_csv('Data/GSE2034.zip', compression='zip', header=0, sep='\t', quotechar='"') # Breast cancer
gse2034_df=raw_gse2034_df.copy()

# preprocess dataset
gse2034_df[['Gene','KEGG-ID']] = gse2034_df['Class'].str.split('#',expand=True)
gse2034_df.drop('Class', inplace=True, axis=1)
cols = gse2034_df.columns.tolist()
cols = cols[-2:] + cols[:-2]
gse2034_df=gse2034_df[cols]

#gse2034_df

In [ ]:
# Change column names (estrogen receptor)
labels=gse2034_df.columns[2:]
for x in range(len(labels)):
    if(labels[x].startswith('ERpos')):
        labels.values[x]="ERpos"
    elif(labels[x].startswith('ERneg')):
        labels.values[x]="ERneg"

In [ ]:
# Get the genes that are present in the GSE2034 dataset and create a dictionary 
# where the keys are the genes and the values are the corresponding KEGG-IDs
# 'a gene can be mapped to more than one Entrez identifier'
gene_list=sorted(set(gse2034_df['Gene'].tolist()))
gene_dict={}
for i in gene_list:
    tmp=(gse2034_df.loc[gse2034_df['Gene'] == i]['KEGG-ID']).copy()
    tmp_list=[]
    for t in tmp:
        tmp_list.append(t)
    gene_dict.update({i:tmp_list})
#print('Gene dictionary (key: Genes, values: KEGG-IDs): '+str(gene_dict))

In [ ]:
# transpose dataframe so that the columns indicate the genes
# and rows correspond to samples (class: ERpos or ERneg)
genes=(gse2034_df['Gene']).copy()
gse2034_df.drop('KEGG-ID', inplace=True, axis=1)
gse2034_df=np.transpose(gse2034_df.iloc[:,1:])
gse2034_df.columns=genes.values.tolist()
#gse2034_df

In [ ]:
# Because one gene might correspond to more than one KEGG-IDs, we calculate the average (or max)
# value and get the following simplified dataframe
gse2034_df=gse2034_df.groupby(level=0,axis=1).mean()
gse2034_df['noProbe']=gse2034_df.mean(axis=1) # Compute 'noProbe' for future use
genes=gse2034_df.columns
gse2034_df

In [ ]:
classes=gse2034_df.index
classes

## 2.1.1. Split data in training and testing sets (size reduction)
- Training set size: 70%
- Testing set size: remaining 30%

In [ ]:
from sklearn.model_selection import train_test_split

training_data, testing_data, training_labels, testing_labels = train_test_split(gse2034_df, gse2034_df.index, test_size = 0.3 ,random_state = 0)

## 2.2. Selected
Cellular processes (15), Signal transduction (Environmental information process) (24), Cancer overview (8).

In [ ]:
raw_selected_df = pd.read_csv('Data/Selected.zip', compression='zip', header=0, sep='\t', quotechar='"')[['SubPathID']]
#raw_selected_df

In [ ]:
# Two types of nodes relations
relations_dict={'Activation':'-->','Inhibition':'--|'}
#relations_dict

In [ ]:
from itertools import chain

# Split each pathway based on the relation
def split_path(data,relation):
    s=[]
    cnt=len(data.split(relations_dict[relation]))
    cnt_tmp=1
    # If there is at least one relation, then split data
    if(cnt>0):
        for e in data.split(relations_dict[relation]):
            if e:
                s.append(e)
                # Remove the final relation
                if(cnt_tmp<cnt):
                    s.append(relations_dict[relation])
                cnt_tmp+=1
    return s

# Make the selected dataset easier to understand, by splitting each pathway based on their nodes and edges
def get_pathway(data):
    s=split_path(data,'Activation')

    for i in range(len(s)):
        tmp_s=split_path(s[i],'Inhibition')
        if(len(s[i])>1):
            s[i]=tmp_s
            
    return list(chain.from_iterable(s))

subpaths_list=[get_pathway(row) for row in raw_selected_df['SubPathID']]
#subpaths_list

In [ ]:
selected_df=pd.DataFrame(subpaths_list).fillna(value=np.nan) # Rows: pathways, Cols: edges and nodes
selected_df

In [ ]:
# Two types of nodes relations
relations_dict={'Activation':'-->','Inhibition':'--|'}

## 2.3. Important values

### 2.3.1. Node genes- all genes of each node

In [ ]:
# Each node in a pathway represents a discrete function mapping to one or more transcript.
# Returns a dictionary corresponding each node of each pathway to its gene or genes.
def Node_genes(df):
    Node_genes={}
    for path in range(df.shape[0]):
        tmp_node=[]
        for node in range(0,len(df.iloc[path][~df.iloc[path].isnull()]),2):
            genes=list(filter(None,[x.strip() for x in df.iloc[path,node].split(' ')]))
            tmp_genes=[]
            for g in genes:
                tmp_genes.append(list(filter(None,[x.strip() for x in g.split('#')]))[0])
            tmp_node.append(tmp_genes)
        Node_genes.update({path:tmp_node})
    return Node_genes

node_genes=Node_genes(selected_df)
node_genes

### 2.3.2. Expression value
For each subpath assign each sample's genes expression values (3D: samples,subpaths,nodes)

In [ ]:
from statistics import mean

# For a specific subpath match the expression values of each gene based on the values in a specific sample.
# In case of multiple genes in one node assign the average value.
def sample_expression_value(subpath,sample):
    sample_expr_val=[]
    for node in range(len(subpath)):
        node_genes=[]
        for gene in range(len(subpath[node])):
            node_genes.append(sample[gene])
        sample_expr_val.append(mean(node_genes)) # Calculate the average expression value of a node's genes
    return sample_expr_val

#sample_no=0
#path_no=0
#sample_expression_value(node_genes[path_no],training_data.iloc[sample_no])

In [ ]:
# For each subpath assign the expression values based on each sample and return a 2D array
def subpath_expression_value(subpath,samples):
    
    # Create a 2D array, where rows are the sample size and columns are the subpath's nodes
    subpath_expr_val=[]
    
    # For each sample
    for sample in range(samples.shape[0]):
        subpath_expr_val.append(list(samples.iloc[sample]))
        
    return subpath_expr_val

#path_no=1
#subpath_expression_value(node_genes[path_no],training_data)

In [ ]:
def get_expression_value(node_genes,samples):
    expr_val={}
    for path in node_genes:
        print(path)
        expr_val.update({path:subpath_expression_value(node_genes[path],samples)})
    return expr_val

#expression_values=get_expression_value(node_genes,training_data)
#expression_values

Because of the time complexity, the expression value is calculated only for one subpath and each training sample

In [ ]:
path_no=0
subpath_expr_val=subpath_expression_value(node_genes[path_no],training_data)
subpath_expr_val

### 2.3.3. P-value and threshold <= 0.05 (gene is significant)
For each gene find the p-value (ttest_ind)

In [ ]:
import scipy.stats as stats

#gse2034_df2=gse2034_df.copy().T # Columns: samples and rows: genes
#gse2034_df2

#### 2.3.3.1. For each gene find cancer and normal mean value and calculate their difference

In [ ]:
#gse2034_df2['ERnegMean']=gse2034_df2['ERneg'].mean(axis=1)
#gse2034_df2['ERposMean']=gse2034_df2['ERpos'].mean(axis=1)
#gse2034_df2['Diffs']=gse2034_df2['ERnegMean']-gse2034_df2['ERposMean']
#gse2034_df2

In [ ]:
def pvalue005(genes):
    pvalArray=np.empty(genes.shape[0])
    pvalArrayUnder005=np.array([])
    indices005=np.array([])
    for x in range(genes.shape[0]):
        st,pval=stats.ttest_ind(genes['ERneg'].iloc[x],genes['ERpos'].iloc[x])
        pvalArray[x]=pval
        if(pval<0.05):
            pvalArrayUnder005=np.append(pvalArrayUnder005,pval)
            indices005=np.append(indices005,x)
    return pvalArray,pvalArrayUnder005,indices005

#pvalList2,pvalListUnder005_2,indices005_2=pvalue005(gse2034_df2)

In [ ]:
#gse2034_df2['P-Value']=pvalList2
#gse2034_df2

In [ ]:
# For each node of a sub-path consisting of more than one genes, get the average value of the p-values.
def path_pvalue(path,df):
    cur_path=[]
    for node in path:
        cur_node=[]
        for gene in node:
            # Check if gene is not in genes' list
            if(not(gene in df.index)):
                cur_node.append(df.loc['noProbe']['P-Value'])
                continue
            cur_node.append(df.loc[gene]['P-Value'])
        cur_path.append(np.mean(cur_node))
    return cur_path
         
def get_pvalues(node_genes,df):
    pvalues={}
    for path in node_genes:
        pvalues.update({path:path_pvalue(node_genes[path],df)})  
    return pvalues

pvalue_threshold=0.05

#pvalues_dict=get_pvalues(node_genes,gse2034_df2) # All sub-paths
#pvalues_dict

#pvalues=path_pvalue(node_genes[path_no],gse2034_df2) # One sub-path
#pvalues

### 2.3.4. Fold Change and Log Fold Change
Add fold change column to genes_df

In [ ]:
# Calculate fold change for each gene
def fold_change(df):
    fc_dict={}
    
    for gene in df.index:
        # Get average value for each case
        erneg_av=df.loc[gene]['ERneg'].mean()
        erpos_av=df.loc[gene]['ERpos'].mean()
        
        # Calculate fold change (B/A)
        cur_fc=erneg_av/erpos_av
        fc_dict.update({gene:cur_fc})
        
    return fc_dict        

#fc=fold_change(gse2034_df2)
#gse2034_df2['Fold Change']=fold_change(gse2034_df2).values()
#gse2034_df2['Log FC']=[math.log(fc+1-min(gse2034_df2['Fold Change'])) for fc in gse2034_df2['Fold Change']] # Calculate log fold change
#gse2034_df2

In [ ]:
# For each node of a sub-path consisting of more than one genes, get the average value of the fold change.
def path_fc(path,df):
    fc=[]
    log_fc=[]
    for node in path:
        cur_fc=[]
        cur_log_fc=[]
        for gene in node:
            # Check if gene is not in genes' list
            if(not(gene in df.index)):
                cur_fc.append(df.loc['noProbe']['Fold Change'])
                cur_log_fc.append(df.loc['noProbe']['Log FC'])
                continue
            cur_fc.append(df.loc[gene]['Fold Change'])
            cur_log_fc.append(df.loc[gene]['Log FC'])
        fc.append(np.mean(cur_fc))
        log_fc.append(np.mean(cur_log_fc))
    return fc,log_fc
            

def get_fc(node_genes,df):
    fc={}
    log_fc={}
    for path in node_genes:
        cur_fc,cur_log_fc=path_fc(node_genes[path],df)
        fc.update({path:cur_fc})  
        log_fc.update({path:cur_log_fc})  
    return fc,log_fc

log_fc_threshold=1.5 

#fc_dict,log_fc_dict=get_fc(node_genes,gse2034_df2) # Return fold change and log fold change in dictionary form for all sub-paths

#fc,log_fc=path_fc(node_genes[path_no],gse2034_df2) # Specific sub-path

### 2.3.5. Extract the DataFrame for future use and create dictionaries for sub-paths based on the previous implemented functions
- Values from 2.3.3. to 2.3.4. are extracted to GSE2034_data.csv so they don't have to be computed every time.
- Create the dictionaries for each subpath using the GSE2034_data file.

In [ ]:
#gse2034_df2.to_csv(r'C:\Users\Foteini Droumalia\Desktop\Φωτεινή Δρουμαλιά\Project\data\GSE2034_data.csv',header=True)
gse2034_df2 = pd.read_csv ('Data\GSE2034_data.csv').iloc[:,1:]
new_columns=list(gse2034_df.index)+list(gse2034_df2.columns[-6:])
gse2034_df2.columns=new_columns
gse2034_df2.index=genes
gse2034_df2

#### 2.3.5.1. P-Value

In [ ]:
pvalues=path_pvalue(node_genes[path_no],gse2034_df2)
pvalues

#### 2.3.5.2. Fold-Change and Log fold-Change

In [ ]:
fc,log_fc=path_fc(node_genes[path_no],gse2034_df2)

### 2.3.6. Differentially Expressed Genes (DEG)

In [ ]:
# Get the differentialy expressed genes as dictionary
def path_de_genes(pvalues):
    de_genes=[]
    for node in range(len(pvalues)):
        if(pvalues[node]<=pvalue_threshold):
            de_genes.append(1)
        else:
            de_genes.append(0)
    return de_genes

def get_de_genes(pvalues_dict):
    de_genes={}
    for path in pvalues_dict:
        de_genes.update({path:path_de_genes(pvalues_dict[path])})
    return de_genes

#de_genes_dict=get_de_genes(pvalues_dict)

de_genes=path_de_genes(pvalues)
de_genes

# 3. Methods

## 3.1. PRS

In [ ]:
from ipynb.fs.full.PRS import *

In [ ]:
#path_PRS(log_fc,fc,pvalues)

## 3.2. MinePath

### 3.2.1. Discretization of gene expression values
Transform gene expression values into high (expressed / up-regulated) or low (not-expressed / down-regulated) gene expression binary equivalents

#### 3.2.1.1. The midpoints between each two consecutive values are calculated;

In [ ]:
def midpoint(num1,num2):
    return (num1+num2)/2

# Returns the midpoints for a specific gene
def gene_midpoints(gene):
    tmp_gene=gene.copy()
    tmp_gene=tmp_gene.reset_index().drop(['index'],axis=1) # Keep previous positions
    
    # The expression values of a gene over the total number of input samples are sorted in descending order;
    sorted_gene=(tmp_gene/tmp_gene.shape[0]).sort_values(by=gene.name,ascending=False)
    
    midpoints_dict={}
    for i in range(sorted_gene.shape[0]-1):
        midpoints_dict.update({sorted_gene.index[i]:midpoint(sorted_gene.iloc[i][gene.name],sorted_gene.iloc[i+1][gene.name])})
    return midpoints_dict

# Returns the midpoints for all the samples
def get_midpoints(df):
    midpoints=[] 
    for gene in range(df.shape[1]):
        midpoints.append(gene_midpoints(df.iloc[:,gene]))
    return midpoints_dict

#### 3.2.1.2. For each midpoint, μi, the Information Gain (IG) of the system is computed. Let IG(S,μi) to denote the IG of the system for midpoint μi.

In [ ]:
import math 

# Calculate the proportion of samples in S that belong in Class C
def P(C,S):
    return list(S).count(C)/len(S)

def E(S,classes,m=1):
    # m not given: calculate the entropy of the system taking into account the prior assignment of sample cases into phenotype classes
    # m given: calculate the respective entropy of the system taking into account its division into subgroups around midpoint μi
    tmp=0
    for c in classes:
        # P(c,S) must be greater than zero
        tmp+=P(c,S)*math.log(P(c,S))/m
    return -(tmp)

# Calculate the Information Gain (IG) of the system
def gene_IG(gene,midpoints):
    classes=sorted(set(gene.index))
    S=gene.index # the samples class
    
    information_gain={}
    for m in midpoints:
        information_gain.update({m:E(S,classes)-E(S,classes,midpoints[m])})
    
    return information_gain

# Calculate the Information Gain (IG) of the system
def IG(df):
    new_df=df
    information_gain={}
    k=0
    for gene in new_df.columns:
        print(k)
        k+=1
        information_gain.update({gene:gene_IG(df[gene])})
    return information_gain

#### 3.2.1.3. The sample cases with expression values lower than the discretization point are assigned the '0' value (meaning that the gene is under-expressed), and the sample cases with expression values bigger that the discretization point are assigned the '1' value (the gene is over-expressed).
The discretization process is applied for each gene separately, and the final dataset is a matrix of discretized, actually binarized, values.

In [ ]:
def discretization_point(midpoints,information_gain):
    # The midpoint with the highest information gain is selected as the discretization point
    max_value=max(information_gain.values())
    max_mid_pos = max(information_gain, key=information_gain.get)
    dis_point=midpoints.get(max_mid_pos)
    return dis_point

def gene_discretization(gene):
    midpoints=gene_midpoints(gene)
    information_gain=gene_IG(gene,midpoints)
    dis_point=discretization_point(midpoints,information_gain)
    
    gene_dis=gene.copy()
    gene_dis[gene_dis<dis_point]=0 # under-expressed
    gene_dis[gene_dis>=dis_point]=1 # over-expressed
    gene_dis=gene_dis.astype('int')
    
    return gene_dis

def discretization(genes):
    dis_genes={}
    k=0
    for gene in genes.columns:
        print(k*100/genes.shape[1])
        k+=1
        dis_genes.update({gene:list(gene_discretization(genes[gene]))})
    return dis_genes

In [ ]:
dis_gse2034_df=pd.DataFrame(discretization(gse2034_df),index=gse2034_df.index)
dis_gse2034_df

### 3.2.2. Functional sub-paths: Matching sub-paths with gene expression profiles
Due to the large volume of data, we refer only to one patient and in the future we will apply the analysis to the rest

In [ ]:
import statistics

# Get the nodes of a specific sub-path in binary form
def sample_functional_subpath(sample,path):
    expr_prof=[]
    for node in path:
        tmp_node=[]
        for gene in node:
            if(~(gene in sample.index)):
                tmp_node.append(sample['noProbe'])
                continue
            tmp_node.append(sample[gene])
        expr_prof.append(statistics.mean(tmp_node))
    return expr_prof

funct_subpath_0=sample_functional_subpath(dis_gse2034_df.iloc[0],node_genes[0])
funct_subpath_0

In [ ]:
# The following functions compute the 'and' and 'xor' boolean operations
def and_boolean_op(num1,num2):
    return num1*num2
    
def xor_boolean_op(num1,num2):
    return 1 if(num1 and not num2) or (not num1 and num2) else 0

In [ ]:
# Calculate the pathway expression of a specific sample with boolean operations
def calc_pathway_expression(path,edges,prev_result):
    operations_dict={'Activation':and_boolean_op,'Inhibition':xor_boolean_op}
    
    # Two types of nodes relations
    relations_dict={'Activation':'-->','Inhibition':'--|'}
    
    if(len(path)>1):
        relation=list(relations_dict.keys())[list(relations_dict.values()).index(edges[1])] # Get the current edge type
        next_node=path[1]
        result=operations_dict[relation](prev_result,next_node)
        calc_pathway_expression(path[2:],edges[1:],result)
    return prev_result

edges=list(selected_df.iloc[0][~selected_df.iloc[0].isnull()][1::2])
calc_pathway_expression(funct_subpath_0[1:],edges,funct_subpath_0[0]) # First sub-path

In [ ]:
# For specific sample and all sub-paths
all_subpaths={}
for path in node_genes:
    edges=list(selected_df.iloc[path][~selected_df.iloc[path].isnull()][1::2])
    funct_subpath=sample_functional_subpath(dis_gse2034_df.iloc[0],node_genes[path])
    all_subpaths.update({path:calc_pathway_expression(funct_subpath[1:],edges,funct_subpath[0])})

In [ ]:
minepath_df=pd.DataFrame(all_subpaths.values(),columns=['Binary Expression'])
minepath_df.to_csv('Results/MinePath.csv',index=False) # Already somputed

In [ ]:
minepath_df = pd.read_csv ('Results/MinePath.csv')
minepath_df